In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Data Loading

In [2]:
joedf = pd.read_csv("us2020/hashtag_joebiden.csv")

C:\Users\barat\anaconda3\envs\envpytorch\lib\site-packages\IPython\core\interactiveshell.py:3186: DtypeWarning: Columns (1,2,3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
trumpdf = pd.read_csv("us2020/hashtag_donaldtrump.csv",lineterminator='\n')

In [8]:
joedf.head()

,created_at,tweet_id,tweet,likes,retweet_count,source,user_id,user_name,user_screen_name,user_description,...,user_followers_count,user_location,lat,long,city,country,continent,state,state_code,collected_at
0,2020-10-15 00:00:01,1.316529221557252e+18,#Elecciones2020 | En #Florida: #JoeBiden dice ...,0.0,0.0,TweetDeck,360666534.0,El Sol Latino News,elsollatinonews,🌐 Noticias de interés para latinos de la costa...,...,1860.0,"Philadelphia, PA / Miami, FL",25.77427,-80.19366,NaN,United States of America,North America,Florida,FL,2020-10-21 00:00:00
1,2020-10-15 00:00:18,1.31652929585929e+18,#HunterBiden #HunterBidenEmails #JoeBiden #Joe...,0.0,0.0,Twitter for iPad,809904438.0,Cheri A. 🇺🇸,Biloximeemaw,"Locked and loaded Meemaw. Love God, my family ...",...,6628.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-21 00:00:00.517827283
2,2020-10-15 00:00:20,1.3165293050069524e+18,@IslandGirlPRV @BradBeauregardJ @MeidasTouch T...,0.0,0.0,Twitter Web App,3494182277.0,Flag Waver,Flag_Wavers,NaN,...,1536.0,Golden Valley Arizona,46.304036,-109.171431,NaN,United States of America,North America,Montana,MT,2020-10-21 00:00:01.035654566
3,2020-10-15 00:00:21,1.3165293080815575e+18,@chrislongview Watching and setting dvr. Let’s...,0.0,0.0,Twitter for iPhone,8.242596012018524e+17,Michelle Ferg,MichelleFerg4,NaN,...,27.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-21 00:00:01.553481849
4,2020-10-15 00:00:22,1.316529312741253e+18,#censorship #HunterBiden #Biden #BidenEmails #...,1.0,0.0,Twitter Web App,1.032806955356545e+18,the Gold State,theegoldstate,A Silicon Valley #independent #News #Media #St...,...,390.0,"California, USA",36.701463,-118.755997,NaN,United States of America,North America,California,CA,2020-10-21 00:00:02.071309132


In [9]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from nltk.stem import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\barat\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\barat\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
joe_final = joedf.drop(['user_name','tweet_id', 'user_id', 'user_screen_name', 'lat', 'long', 'collected_at','state_code', 'user_location', 'user_join_date', 'source', 'user_followers_count', 'user_description', 'city'], axis = 1)
trump_final = trumpdf.drop(['user_name','tweet_id', 'user_id', 'user_screen_name', 'lat', 'long', 'collected_at','state_code','user_location', 'user_join_date', 'source', 'user_followers_count', 'user_description', 'city'], axis = 1)

In [7]:
joe_final = joe_final[joe_final['tweet'].notnull()]
trump_final = trump_final[trump_final['tweet'].notnull()]

### Hugging Face Sentiment Analysis

In [10]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

### Testing model

In [11]:
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
#model.save_pretrained(MODEL)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
text = "Covid cases are increasing fast!"
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)
# Print labels and scores
ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = config.id2label[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

1) negative 0.7236
2) neutral 0.2287
3) positive 0.0477


In [5]:
text = "Covid cases are increasing fast!"
text = preprocess(text)
text

'Covid cases are increasing fast!'

In [27]:
joetest = joe_final['tweet']

### Preprocessing

In [29]:
import re
import emoji
from langdetect import detect, LangDetectException

def clean_tweet(tweet):
    # Convert emojis to text
    tweet = emoji.demojize(tweet, delimiters=("", " "))
    # Remove URLs
    tweet = re.sub(r"http\S+|www\S+|https\S+", '', tweet, flags=re.MULTILINE)
    # Optionally, decide whether to remove @ and # based on your need
    tweet = re.sub(r'\@\w+|\#','', tweet)
    # Remove unwanted punctuation but retain '.', ',', '?', '!', and ':'
    tweet = re.sub(r'[^\w\s\.\,\?\!\:]', '', tweet)
    # Remove numbers if they are not relevant
    tweet = re.sub(r'\d+', '', tweet)
    # Normalize whitespace
    tweet = re.sub(r'\s+', ' ', tweet).strip()
    return tweet
def is_english(text):
    try:
        return detect(text) == 'en'
    except LangDetectException:
        return False
    
joetest = joetest.apply(clean_tweet)
joetest = joetest[joetest.apply(is_english)]

### Predicting Labels

In [94]:
import tqdm
resultbiden = []

for text in tqdm.tqdm(joetest):
    text = preprocess(text)
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    resultbiden.append(config.id2label[np.argmax(scores)])

100%|██████████████████████████████████████████████████████████████████████████████████| 87/87 [00:07<00:00, 10.93it/s]
